In [ ]:
# Implementation of a very stripped down version of the C program aigsim

# Source Files
modelFile = 'examples/aigTestSMV2.aag.txt'
stimFile  = 'examples/stim1.txt'
# modelFile = 'aigTestLatch.aag.txt'
# stimFile  = 'stimELatch.txt'

STORE_PATH = ''

validInput = {"0","1"}


In [ ]:
from dataclasses import dataclass
import math
import re

class aiger_symbol:

    lit        = 0  # Number assigned to this gate by the model file, format: [0..2*maxvar+1]
    gID        = 0  # Gate ID number. Assumed use is to sequentially number gates of a same class
    type       = '' # What type of object is this 
    gName      = '' # Internally generated short name of gate
    modName    = '' # Name give to the gate by the model file
    myInput    = '' # The gate that feeds this one (builds a binary tree of the model gates)
    myInputNeg = False # Is the feeder gate negated
    curVal     = 0  # The current gate value
    statesSeen = 0  # Keeps a running list of which input states the gate has seen
	
    def __init__(self,lit,typeName,gID=0,gName=''):
        self.lit = lit
        self.type = typeName
        if self.gName == '':
            self.gName = typeName[0] + str(gID)
 
    def connect(self,gateList):
        self.myInput = gateList[int(self.lit/2)]
        if self.lit % 2 != 0:
            self.myInputNeg = True
            
    def resetGate(self):
        pass
          
    def setModName(self,mName):
        self.modName = mName
        
    def prepStep(self):
        self.oldVal = self.curVal
        self.curVal = float('nan')
          
    def step(self):
        return(self.curVal)
        
    def printSelf(self):
        conStr = self.myInput.gName
        if self.myInputNeg == True:
           conStr += '*'
        else:
           conStr += ' '
           
        print('Type: {:6} lit: {:3}                      input: {:4}      name:{:4} {:20}'.format(self.type,self.lit,conStr,self.gName,self.modName))                                


In [ ]:
class aiger_const(aiger_symbol):

    def prepStep(self):
        self.oldVal = self.curVal

In [ ]:
class aiger_input(aiger_symbol):

    controlled = False
    
    def setModName(self,mName):
        super().setModName(mName)
        
        if re.search('control', mName, re.IGNORECASE) and not re.search('uncontrol', mName, re.IGNORECASE):
            self.controlled = True

In [ ]:
class aiger_output(aiger_symbol):

          
    def step(self):
        self.curVal = self.myInput.step()
        return(self.curVal)

In [ ]:
# - Note this code does not support reset reset values outside {0,1}.

class aiger_latch(aiger_symbol):

    next   = 0  # Number of gate connected to this gate's input as literal - latches only
    reset  = 0  # The initial state of the latch upon reset/startup
    nextVal = 0

    def __init__(self,lit,input,reset,gID=0,gName=''):
        super().__init__(lit,'Latch',gID,gName)
        self.next = input
        self.reset = reset
        self.curVal = reset
        self.nextVal = reset
        
    def connect(self,gateList):
        self.myInput = gateList[int(self.next/2)]
        if self.next % 2 != 0:
            self.myInputNeg = True

    def resetGate(self):
        self.curVal = self.reset
        self.nextVal = self.reset
        
    def prepStep(self):
        self.oldVal = self.curVal
        self.curVal = float('nan')

    def step(self):
        if math.isnan(self.curVal):
            self.curVal  = self.nextVal

            self.nextVal = self.myInput.step()
            if self.myInputNeg == True:
                self.nextVal = int(bin(self.nextVal+1)[-1])
                
            self.statesSeen = self.statesSeen | 0x1 << self.curVal
            
        return self.curVal
        
    def printSelf(self):
        conStr = self.myInput.gName
        if self.myInputNeg == True:
           conStr += '*'
        else:
           conStr += ' '
           
        print('Type: {:6} lit: {:3} next: {:3} reset: {:3} input: {:4}      name:{:4} {:20}'.format(self.type,self.lit,self.next,self.reset,conStr,self.gName,self.modName))                                


In [ ]:
class aiger_and(aiger_symbol):
    
    rhs0 = 0  # as literal [0..2*maxvar+1]
    rhs1 = 0  # as literal [0..2*maxvar+1] 
    in0  = 0
    in0Neg = False
    in1  = 0
    in1Neg = False

    def __init__(self,lit,rhs0,rhs1,gID=0,gName=''):
        super().__init__(lit,'And',gID,gName)
        self.rhs0 = rhs0
        self.rhs1 = rhs1

    def connect(self,gateList):
        self.in0 = gateList[int(self.rhs0/2)]
        if self.rhs0 % 2 != 0:
            self.in0Neg = True

        self.in1 = gateList[int(self.rhs1/2)]
        if self.rhs1 % 2 != 0:
            self.in1Neg = True

    def step(self):
        if math.isnan(self.curVal):
            rhs0 = self.in0.step()
            if self.in0Neg == True:
                rhs0 = int(bin(rhs0+1)[-1])
            rhs1 = self.in1.step()
            if self.in1Neg == True:
                rhs1 = int(bin(rhs1+1)[-1])
            self.curVal = rhs0 & rhs1

            if rhs1 == 0 and rhs0 == 0:
                self.statesSeen = self.statesSeen | 0x1
            elif rhs1 == 0 and rhs0 == 1:
                self.statesSeen = self.statesSeen | 0x2
            elif rhs1 == 1 and rhs0 == 0:
                self.statesSeen = self.statesSeen | 0x4
            elif rhs1 == 1 and rhs0 == 1:
                self.statesSeen = self.statesSeen | 0x8
          
        return self.curVal

    def printSelf(self):        
        conStr = self.in0.gName
        if self.in0Neg == True:
           conStr += '*'
        
        conStr += ' '*(5-len(conStr))

        conStr += self.in1.gName
        if self.in1Neg == True:
           conStr += '*'
        
        conStr += ' '*(9-len(conStr))

        print('Type: {:6} lit: {:3} rhs0: {:3}  rhs1: {:3} input: {:9} name:{:10}'.format(self.type,self.lit,self.rhs0,self.rhs1,conStr,self.gName))


In [ ]:
import numpy as np

class aigTransionTable:

    tTable = []
    bTable = []
    
    def __init__(self,numLatches,numInputs):
    
        self.tTable = np.full((pow(2,numLatches),pow(2,numInputs)),float('nan'))
        self.bTable = np.full(pow(2,numLatches),float('nan'))
    	
    def updateTransTable(self,curState, nextState, stim, bad):
        
        if np.isnan(self.tTable[curState,stim]) == True:
        	self.tTable[curState,stim] = nextState
        	
        # Flag inconsistencies where the same transition goes to a different next state
        elif self.tTable[curState,stim] != nextState:
        	self.tTable[curState,stim] = -1
        	
        if bad > 0:
            self.bTable[curState] = 1

    def printTable(self,trim=True):

        inputLen = len(bin(self.tTable.shape[1]-1)[2:])
        stateLen = len(str(self.tTable.shape[0]))
        
        if stateLen > inputLen:
            colWidth = stateLen
        else:
            colWidth = inputLen
            
        print('\nTransistion Table')   
        print('------------------')    
        print('\n        Input')
        print('     ','-'*(colWidth + 1)*self.tTable.shape[1])
         
        print('State ',end='')
        for i in range(self.tTable.shape[1]-1,-1,-1):
            strOut = '0'* (inputLen - len(bin(i)[2:])) + bin(i)[2:]
            print(' {:>{width}}'.format(strOut,width=inputLen),end='')
        print('')  
        print('-----','-'*(colWidth + 1)*self.tTable.shape[1])
        
        for i in range(self.tTable.shape[0]):
            outStr = '{:3d}   '.format(i)
            visitCnt = 0
            for j in range(self.tTable.shape[1]-1,-1,-1):
                if np.isnan(self.tTable[i,j]) == True:
                    outStr += ' {:>{width}}'.format('.',width=colWidth)
                else:
                    outStr += ' {:>{width}d}'.format(int(self.tTable[i,j]),width=colWidth)
                    visitCnt += 1
                    
            if not(visitCnt == 0 and trim == True) :
                print(outStr,' {:4d}'.format(visitCnt))      
        
    # For more on DOT file format see:
    # https://www.graphviz.org/doc/info/attrs.html#k:arrowType
    # https://www.graphviz.org/doc/info/lang.html
    
    def printDotFile(self,outfile):
        
        f = open(outfile,"w")

        f.write('digraph "{}" {{\n'.format(outfile))
        
        inputLen = len(bin(self.tTable.shape[1]-1)[2:])
        for i in range(self.tTable.shape[0]):
            visitCnt = 0
            for j in range(self.tTable.shape[1]):
                if np.isnan(self.tTable[i,j]) != True:
                    visitCnt += 1
            if visitCnt > 0:
                if self.bTable[i] > 0:
                    outstr = ('{:}[shape=circle,color=red];\n'.format(i))
                else:
                    outstr = ('{:}[shape=circle,color=blue];\n'.format(i))
                f.write(outstr)

        for i in range(self.tTable.shape[0]):
            outstr = ''
            for j in range(self.tTable.shape[1]):
                if np.isnan(self.tTable[i,j]) != True:
                    lbl = '0'* (inputLen - len(bin(j)[2:])) + bin(j)[2:]
                    outstr += ('{:} -> {:} [label="{:}"];\n'.format(i, int(self.tTable[i,j]),lbl))
            if len(outstr) > 0:
                f.write(outstr)
        
        f.write('}\n')
        f.close()

In [ ]:
class Reader:
    
    inFile = ''
    
    def _init_(self):
        pass

#--------------------------------------------------------------------------------------
    
    def openFile(self,file):
        
        self.inFile = open(file)

#--------------------------------------------------------------------------------------

    def procModelNames(self,model):
    
        for line in self.inFile:
            gateNames = line.split(' ',1)
            if gateNames[0][0] != 'c':
                if gateNames[0][0] == 'i':
                    model.inputs[int(gateNames[0][1:])].setModName(gateNames[1].rstrip())
                    
                elif gateNames[0][0] == 'l':
                    model.latches[int(gateNames[0][1:])].setModName(gateNames[1].rstrip())
                
                elif gateNames[0][0] == 'o':
                    model.outputs[int(gateNames[0][1:])].setModName(gateNames[1].rstrip())

            else:
                break

        
#--------------------------------------------------------------------------------------
        
    def readHeader(self,model):
        
        args = (self.inFile.readline()).split()
        
        if args[0] != 'aag':
            return -1
        
        if len(args) < 6:
            sys.exit('Insufficient model parameters MILOA minimum requirement')
            
        model.maxvar      = int(args[1])
        model.num_inputs  = int(args[2])
        model.num_latches = int(args[3])
        model.num_outputs = int(args[4])
        model.num_ands    = int(args[5])
        
        if len(args) >= 7:
            model.num_bad = int(args[6])
            
        if len(args) >= 8:
            model.num_constraints = int(args[7])
       
        if len(args) >= 9:
            model.num_justice = int(args[8])

        if len(args) >= 10:
            model.num_fairness = int(args[9])

        return 0

#--------------------------------------------------------------------------------------
    
    def validateInput(self,numArgs,errStr,verbose):
        
        args = (self.inFile.readline()).split()
        
        err = 0
        if len(args) < numArgs:
            print(errStr)
            err = -1
        
        if verbose == True:
            print(args)
            
        return args,err

#--------------------------------------------------------------------------------------

    def readModel(self,model):
        
        verbose = False
        gateList = [0] * (model.maxvar + 1)
        gateList[0] = aiger_const(0,'Constant',0)
        
        model.inputs = [0]*model.num_inputs
        for i in range(0,model.num_inputs):
            args,err = self.validateInput(1,'Invalid model definition - Input',verbose)
            if err == 0:
                model.inputs[i] = aiger_input(int(args[0]),'Input',i)
                gateList[int(int(args[0])/2)] = model.inputs[i]
                 
        model.latches = [0]*model.num_latches
        for i in range(0,model.num_latches):
            args,err = self.validateInput(2,'Invalid model definition - Latches',verbose)
            if err == 0:
                if len(args) == 2:
                    args.append('0')
                model.latches[i] = aiger_latch(int(args[0]),int(args[1]),int(args[2]),i)
                gateList[int(int(args[0])/2)] = model.latches[i]
        
        model.outputs = [0]*model.num_outputs
        for i in range(0,model.num_outputs):
            args,err = self.validateInput(1,'Invalid model definition - Output',verbose)
            if err == 0:
                model.outputs[i] = aiger_output(int(args[0]),'Output',i)
        
        model.bad = [0]*model.num_bad
        for i in range(0,model.num_bad):
            args,err = self.validateInput(1,'Invalid model definition - Bad State',verbose)
            if err == 0:
                model.bad[i] = aiger_output(int(args[0]),'Bad',i)
            
        model.constraint = [0]*model.num_constraints
        for i in range(0,model.num_constraints):
            args,err = self.validateInput(1,'Invalid model definition - Constraints',verbose)
            if err == 0:
                model.constraint[i] = aiger_output(int(args[0]),'Const',i)

        # Read but ignore any justice properties 
        if model.num_justice > 0:
            justiceSizes = [0]*model.num_justice
            for j in range(0,model.num_justice):
                justiceSizes[j] = int((self.inFile.readline()).split()[0])

            for j in range(0,model.num_justice):
                for k in range(0,justiceSizes[j]):
                    tmp1 = (self.inFile.readline()).split()

        model.ands = [0]*model.num_ands
        for i in range(0,model.num_ands):
            args,err = self.validateInput(3,'Invalid model definition - Ands',verbose)
            if err == 0:
                model.ands[i] = aiger_and(int(args[0]),int(args[1]),int(args[2]),i)
                gateList[int(int(args[0])/2)] = model.ands[i]

        # Connect all the gate inputs up
        for i in range(0,model.num_inputs):
            model.inputs[i].connect(gateList)
  
        for i in range(0,model.num_latches):
            model.latches[i].connect(gateList)

        for i in range(0,model.num_outputs):
            model.outputs[i].connect(gateList)

        for i in range(0,model.num_bad):
            model.bad[i].connect(gateList)

        for i in range(0,model.num_constraints):
            model.constraint[i].connect(gateList)
		
        for i in range(0,model.num_ands):
            model.ands[i].connect(gateList)
            
        self.procModelNames(model)
        
        #Get a count of the controllable InputStates
        for i in range(0,model.num_inputs):
            if model.inputs[i].controlled == True:
                model.num_inputsCtl += 1
                
#--------------------------------------------------------------------------------------
              
    def getStim(self):
        
        args = (self.inFile.readline()).split()
        
        return args

#--------------------------------------------------------------------------------------


In [ ]:
class Model:

    stepNum         = 0
    maxvar          = 0
    num_inputs      = 0
    num_inputsCtl   = 0
    num_latches     = 0
    num_outputs     = 0
    num_ands        = 0
    num_bad         = 0
    num_constraints = 0
    num_justice     = 0
    num_fairness    = 0

    inputs     = [] # [0..num_inputs]
    latches    = [] # [0..num_latches]
    outputs    = [] # [0..num_outputs]
    bad        = [] # [0..num_bad]
    constraint = [] # [0..num_constraints]
    ands       = [] # [0..num_ands]

    def _init_(self):
        pass
    
    def initModel(self):
        self.stepNum = 0
        self.transTable = aigTransionTable(self.num_latches,self.num_inputs)
        for i in range(0,self.num_latches):
            self.latches[i].resetGate()
                    
    def validateInput(self,args):
        
        err = 0
        if len(args) == self.num_inputs:
            current = [0] * self.num_inputs
            for i in range (self.num_inputs):
                if validInput.issuperset(args.rstrip()):   
                    current[i] = int(args[i])
                else:
                    print('invalid characters in input string')
                    err = -1
                    
        else:
            sys.exit('invalid input string length. Expected {:d} Got {:d} {:s}'.format(self.num_inputs,len(args),args))

        return current,err
        
    def getCurVal(self,lit):
        val = self.current[int(lit/2)]
        if lit%2 != 0:
            val = int(bin(val+1)[-1])
        
        return val
            
    def step(self,args):
        
        for i in range(0,self.num_inputs):
            self.inputs[i].prepStep()
            
        for i in range(0,self.num_latches):
            self.latches[i].prepStep()

        for i in range(0,self.num_ands):            
            self.ands[i].prepStep()
        
        stim,err = self.validateInput(args)  

        # Process the input stimuli
        for i in range(0,self.num_inputs):
            self.inputs[i].curVal = stim[i]

        # Process the latches
        curState  = ''
        nextState = ''
        for i in range(0,self.num_latches):
            self.latches[i].step()
            curState += ("{:1d}".format(self.latches[i].curVal))
            nextState += ("{:1d}".format(self.latches[i].nextVal))
        curState  = int(curState,2)
        nextState = int(nextState,2)
        
        # Process the and gates
        for i in range(0,self.num_ands):
            self.ands[i].step()
            
         # Process the output gates
        for i in range(0,self.num_outputs):
            self.outputs[i].step()

         # Process the bad states
        bad = 0
        for i in range(0,self.num_bad):
            self.bad[i].step()
            if self.bad[i].curVal > 0:
                bad = 1

         # Process the constrained states (negative logic -> 1 = constraint holds)
        constraint = 0
        for i in range(0,self.num_constraints):
            self.constraint[i].step()
            if self.constraint[i].curVal == 0:
                constraint = 1

        self.stepNum += 1
        
        self.transTable.updateTransTable(curState,nextState,int(args,2),bad)
        
        return self.stepNum
        
    def printSelf(self):
        print('Model')
        print('-----')
        print('maxvar          = ',self.maxvar)
        print('num_inputs      = ',self.num_inputs)
        print('  controlled    = ',self.num_inputsCtl)
        print('num_latches     = ',self.num_latches)
        print('num_outputs     = ',self.num_outputs)
        print('num_bad         = ',self.num_bad)
        print('num_constraints = ',self.num_constraints)
        print('num_ands        = ',self.num_ands)
        
        for i in range(0,self.num_inputs):
            self.inputs[i].printSelf()
            
        for i in range(0,self.num_latches):
            self.latches[i].printSelf()
            
        for i in range(0,self.num_outputs):
            self.outputs[i].printSelf()
            
        for i in range(0,self.num_bad):
            self.bad[i].printSelf()
            
        for i in range(0,self.num_constraints):
            self.constraint[i].printSelf()
                        
        for i in range(0,self.num_ands):
            self.ands[i].printSelf()
        
    def printState(self,pOptions,stepNum=0):
    
        status = self.getState()
        
        outStr = self.stateStr(pOptions[1],pOptions[2])
        
        if pOptions[0] == True:
            outStr = "{:4d} ".format(stepNum) + outStr
                        
        print(outStr)

    def stateStr(self,pAStates = True,pHistory = True):
    
        status = self.getState()

        statusStr = ''
        
        statusStr += status["latches_now"]
        statusStr += ' '
        statusStr += status["inputs"]
        statusStr += ' '
        statusStr += status["outputs"]
        statusStr += ' '
        statusStr += status["bad"]
        statusStr += ' '
        statusStr += status["constraint"]
        statusStr += ' '
        statusStr += status["latches_next"]
        statusStr += ' '
        
        if pAStates == True:
            statusStr += status["ands"]
            statusStr += ' '
            
        if pHistory == True:
            statusStr += status["latches_seen"]
            statusStr += ' '
            
            statusStr += status["states_seen"]
       
        return statusStr

    def getState(self):
    
        states = {}
        
        statusStr = ''
        for i in range(0,self.num_latches):
            statusStr += ("{:1d}".format(self.latches[i].curVal))
        states['latches_now'] = statusStr

        statusStr = ''
        for i in range(0,self.num_inputs):
            statusStr += ("{:1d}".format(self.inputs[i].curVal))
        states['inputs'] = statusStr
            
        statusStr = ''
        for i in range(0,self.num_outputs):
            statusStr += ("{:1d}".format(self.outputs[i].curVal))
        states['outputs'] = statusStr
            
        statusStr = ''
        for i in range(0,self.num_bad):
            statusStr += ("{:1d}".format(self.bad[i].curVal))
        states['bad'] = statusStr
            
        statusStr = ''
        for i in range(0,self.num_constraints):
            statusStr += ("{:1d}".format(self.constraint[i].curVal))
        states['constraint'] = statusStr

        statusStr = ''
        for i in range(0,self.num_latches):
            statusStr += ("{:1d}".format(self.latches[i].nextVal))
        states['latches_next'] = statusStr
            
        statusStr = ''
        for i in range(0,self.num_ands):
            statusStr += ("{:1d}".format(self.ands[i].curVal))            
        states['ands'] = statusStr
            
        statusStr = ''
        for i in range(0,self.num_latches):
            statusStr += ("{:02b}".format(self.latches[i].statesSeen))
        states['latches_seen'] = statusStr
            
        statusStr = ''
        for i in range(0,self.num_ands):
            statusStr += ("{:04b}".format(self.ands[i].statesSeen,''))
        states['states_seen'] = statusStr
       
        return states

    def getStats(self):
        
        stats = {}
        stats['maxvar'] = self.maxvar
        stats['inputs'] = self.num_inputs
        stats['controlled'] = self.num_inputsCtl
        stats['latches'] = self.num_latches
        stats['outputs'] = self.num_outputs
        stats['bad']     = self.num_bad
        stats['constraing'] = self.num_constraints
        stats['ands']    = self.num_ands
        
        return stats


In [ ]:
verbose0 = True
verbose1 = True
printSM  = False
printDot = False
pOptions = [False] * 3

model = Model()

reader = Reader()
reader.openFile(modelFile)
reader.readHeader(model)
reader.readModel(model)

if verbose0 == True:
    model.printSelf()

model.initModel()

reader = Reader()
reader.openFile(stimFile)

done = False

while done != True:
    stim = reader.getStim()
    if len(stim) > 0:
        if stim[0] == '.':
            done = True
        else:
            stepNum = model.step(stim[0])
            if verbose1 == True:
                model.printState(pOptions,stepNum)

    else:
        print('Stim file not properly terminated. Last line should only contain a period')
        done = True

if printSM == True:
    model.transTable.printTable()

if printDot == True:
    model.transTable.printDotFile(os.path.splitext(modelFile)[0] + '.dot')



In [ ]:
a = 1
b = bin(a+1)[-1]
print(a+1)
print(int(b))
print(type(b))
print(b[-1])

In [ ]:
import math

x = float('nan')
math.isnan(x)
